In [13]:
import numpy as np
import geopandas as gpd
import time
import utils.analysis as an

from rasterio.transform import from_origin
import rasterio

# Zonas Medellin

In [ ]:
# Load the point shapefile
zo = ['aguacatala_HL','floresta_HH','moravia_LH']
z = zo[-1]
for z in zo:
    print(f"Procesando zona: {z}")  # Print which zone you are processing
    nodes = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_nodes_proj_net_initial.shp')
    edges = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_edges_proj_net_initial.shp')
    nodes = nodes.to_crs('epsg:32618')
    edges = edges.to_crs('epsg:32618')

    nodes_tess = gpd.read_file(f'../data/output/shape/network_tessellations/{z}/{z}_tessellations_nodes.shp')
    edges_tess = gpd.read_file(f'../data/output/shape/network_tessellations/{z}/{z}_tessellations_edges.shp')
    nodes_tess = nodes_tess.to_crs('epsg:32618')
    edges_tess = edges_tess.to_crs('epsg:32618')

    # Start timer
    start_time = time.time()

    # Run the function
    nodes_tess, mesh_gdf, x_min, y_min, x_max, y_max = an.calculate_density(nodes_tess, bandwidth=80, pixel_size=5, kernel_shape='quartic')

    # Extraer coordenadas de los puntos de la malla de densidad
    mesh_coords = np.array(list(zip(mesh_gdf.geometry.x, mesh_gdf.geometry.y)))
    puntos_coords = np.array(list(zip(nodes.geometry.x, nodes.geometry.y)))
    # Construir el árbol KD con las coordenadas de la malla de densidad
    from scipy.spatial import cKDTree
    tree = cKDTree(mesh_coords)
    # Buscar los índices de los puntos más cercanos en la malla de densidad
    _, indices = tree.query(puntos_coords, k=1)
    # Asignar los valores de densidad correspondientes
    nodes['density'] = mesh_gdf.iloc[indices]['density'].values

    # Make sure 'osmid' is the index on the GeoDataFrame 'nodes'
    nodes = nodes.set_index('osmid')

    # Assign density to each edge as the average of nodes 'u' and 'v'
    for i in edges.index:
        u = edges.loc[i, 'u']  # Get edge node 'u'
        v = edges.loc[i, 'v']  # Get edge node 'v'
        
        # Verify that both nodes 'u' and 'v' exist in the GeoDataFrame 'nodes'
        if u in nodes.index and v in nodes.index:
            # Calculate the average density between the two nodes connected by the edge
            edges.loc[i, 'den_inter'] = (nodes.loc[u, 'density'] + nodes.loc[v, 'density']) / 2
        else:
            # If any of the nodes do not have density, assign NaN or a default value
            edges.loc[i, 'den_inter'] = np.nan

    # Calculate total elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"El proceso tomó {elapsed_time/60:.2f} minutos.\n")

    # # Call the function with the density result
    # an.plot_density(nodes, density)

    # Save modified nodes to a shapefile
    output_nodes_shapefile_path = (f'../data/output/shape/physical_variables/intersections/{z}/{z}_intersections_nodes.shp')
    nodes = nodes.to_crs('epsg:4326')
    nodes = nodes.rename(columns={'density':'den_inter'})
    nodes.to_file(output_nodes_shapefile_path)

    # Save modified edges to a shapefile
    output_edges_shapefile_path = (f'../data/output/shape/physical_variables/intersections/{z}/{z}_intersections_edges.shp')
    edges = edges.to_crs('epsg:4326')
    edges.to_file(output_edges_shapefile_path)

    # # Save the density map in raster format (GeoTIFF)
    # raster_output_path = (f'../output/shape/heatmaps/{z}/{z}_heatmap.tif')
    # transform = from_origin(x_min, y_min + (y_max - y_min), 5, -5)  # Change pixel size in Y to negative

    # # Save the raster using rasterio
    # with rasterio.open(raster_output_path, 'w', driver='GTiff', height=density.shape[0], width=density.shape[1],
    #                    count=1, dtype='float32', crs='EPSG:32618', transform=transform) as dst:
    #     dst.write(density, 1)

Procesando zona: aguacatala_HL
El proceso tomó 2.58 minutos.

Procesando zona: floresta_HH
El proceso tomó 4.21 minutos.

Procesando zona: moravia_LH
El proceso tomó 3.35 minutos.

